In [1]:
#Importo la librerias
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import tree

from sklearn.model_selection import train_test_split 
import sklearn

In [2]:
#importo el dataset y hago revision de los datos

df = pd.read_csv("titles_filtrado.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5850 entries, 0 to 5849
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            5850 non-null   int64  
 1   id                    5850 non-null   object 
 2   title                 5849 non-null   object 
 3   type                  5850 non-null   object 
 4   description           5832 non-null   object 
 5   release_year          5850 non-null   int64  
 6   age_certification     3231 non-null   object 
 7   runtime               5850 non-null   int64  
 8   genres                5850 non-null   object 
 9   production_countries  5850 non-null   object 
 10  seasons               2106 non-null   float64
 11  imdb_id               5447 non-null   object 
 12  imdb_score            5850 non-null   float64
 13  imdb_votes            5352 non-null   float64
 14  tmdb_popularity       5759 non-null   float64
 15  tmdb_score           

In [3]:
# Drop de columnas innecesarias
df.drop("id", inplace=True, axis=1)

df.drop("imdb_id", inplace=True, axis=1)

# Limpio `genres` y `production_countries`, devuelta tomando el primer elemento de de la lista que cada valor retenga
def parse_list(row):
    parsed = str(row)[1:-1].split(",")
    for i in range(len(parsed)):
        parsed[i] = parsed[i].strip()[1:-1]
        
    return parsed[0] if parsed[0] != '' else 'N/A'

df['genres_parsed'] = df['genres'].map(parse_list)
df['prod_countries_parsed'] = df['production_countries'].map(parse_list)
df[['genres_parsed', 'prod_countries_parsed']]

,genres_parsed,prod_countries_parsed
0,documentation,US
1,drama,US
2,drama,US
3,fantasy,GB
4,war,GB
...,...,...
5845,romance,NG
5846,drama,N/A
5847,comedy,CO
5848,N/A,US


In [4]:
# Añado la columna 'is_movie'
df['is_movie'] = df.type.map(lambda x: 0 if x == 'SHOW' else 1)

In [5]:
# Obtengo todos los valores nulos
df.isnull().sum()

Unnamed: 0                  0
title                       1
type                        0
description                18
release_year                0
age_certification        2619
runtime                     0
genres                      0
production_countries        0
seasons                  3744
imdb_score                  0
imdb_votes                498
tmdb_popularity            91
tmdb_score                  0
genres_parsed               0
prod_countries_parsed       0
is_movie                    0
dtype: int64

In [6]:
# Drop de columnas innecesarias
df.drop("title", axis=1, inplace=True)
df.drop("description", axis=1, inplace=True)

# Relleno valores nulos
df['age_certification'].fillna("N/A", inplace=True)
df['seasons'].fillna(0.0, inplace=True)
df['tmdb_popularity'].fillna(df['tmdb_popularity'].mean(), inplace=True)
df['imdb_votes'].fillna(df['imdb_votes'].mean(), inplace=True)


# Confirmo que no queden valores nulos
df.isnull().sum()

Unnamed: 0               0
type                     0
release_year             0
age_certification        0
runtime                  0
genres                   0
production_countries     0
seasons                  0
imdb_score               0
imdb_votes               0
tmdb_popularity          0
tmdb_score               0
genres_parsed            0
prod_countries_parsed    0
is_movie                 0
dtype: int64

In [7]:
# Trasnfroma las variables categoricas en numericas
all_data = pd.get_dummies(df[['age_certification', 'genres_parsed', 'prod_countries_parsed', 'imdb_votes', 'seasons', 'is_movie']])



In [8]:
from sklearn.model_selection import train_test_split, cross_val_score

X = all_data.copy()
y = df.pop('imdb_score')




In [9]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn import utils


#Convierto los valores de Y a valores categoricos


lab = preprocessing.LabelEncoder()
y_transformed = lab.fit_transform(y)


X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size=0.2, random_state = 42)

In [10]:
#Creamos un arbol de decisión sencillo y lo fiteamos
tree_class = DecisionTreeClassifier(random_state=None)
tree_class.fit(X_train, y_train)

DecisionTreeClassifier()

In [ ]:

y_test_pred = tree_class.predict(X_test) #Prediccion en Test

In [11]:
from sklearn.metrics import accuracy_score

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('% de aciertos sobre el set de evaluación:',test_accuracy)

NameError: name 'y_test_pred' is not defined

Como se ve. Con un algoritmo de Random Forest se llega a un 10% de exactitud solamente. Por lo que debo optar por otro algoritmo



Elijo uno de regresion lineal

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

#Creo las variables

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [19]:
#Entrenamiento del modelo
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [23]:
#Hacemos nuestras predicciones
y_pred = regressor.predict(X_test)

In [ ]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df

,Actual,Predicted
2089,7.0,6.749619
4909,5.6,5.973346
2961,5.8,5.986581
2070,5.9,6.135598
3054,7.3,7.426295
...,...,...
2457,7.5,6.116354
1837,8.2,6.378707
2377,5.8,5.873893
4949,5.3,5.440751


In [22]:
linear_accuracy = accuracy_score(y_test, y_test_pred)

ValueError: Found input variables with inconsistent numbers of samples: [1755, 1170]